In [ ]:
import tobias
import pyBigWig, pandas as pd, numpy as np, matplotlib.pyplot as plt
from pathlib import Path

In [ ]:
plt.rcParams.update({
    "figure.figsize": (10, 5),
    "axes.spines.top": False,
    "axes.spines.right": False,
    "axes.grid": False,
    "axes.titlesize": 12,
    "axes.labelsize": 10,
    "xtick.labelsize": 9,
    "ytick.labelsize": 9,
})

In [ ]:
def read_bw(bw_path: str, chrom: str, start: int, end: int) -> np.ndarray:
    """Read a bigWig segment as a numpy array, fill NaNs with 0."""
    with pyBigWig.open(bw_path) as bw:
        arr = np.array(bw.values(chrom, start, end))
    arr[np.isnan(arr)] = 0
    return arr

def load_sites(bed_path: str, chrom: str, start: int, end: int) -> pd.DataFrame:
    """Read BED (≥3 cols), return only sites overlapping [start, end) on chrom."""
    cols = ["chrom","start","end","name","score","strand"]
    df = pd.read_csv(bed_path, sep="\t", comment="#", header=None,
                     names=cols, usecols=[0,1,2,3,4,5], engine="python")
    return df.query("chrom == @chrom and start < @end and end > @start").copy()

def pick_region_from_bed(bed_path: str, keyword: str, pad: int = 300):
    """Pick a window centered on the first site whose name contains keyword."""
    cols = ["chrom","start","end","name","score","strand"]
    df = pd.read_csv(bed_path, sep="\t", comment="#", header=None, names=cols, engine="python")
    hit = df[df["name"].astype(str).str.contains(keyword, case=False, na=False)]
    r = hit.iloc[0] if not hit.empty else df.iloc[0]
    center = (int(r.start) + int(r.end)) // 2
    chrom = str(r.chrom)
    return chrom, max(0, center - pad), center + pad

In [ ]:
def plot_chromatin_tracks(
    region,                          # (chrom, start, end)
    signal_bigwigs,                  # list of paths (e.g., corrected ATAC)
    footprint_bigwigs=None,         # list of paths (or None)
    sites_bed=None,                 # BED with TFBS (optional but recommended)
    labels=None,                    # labels for legend, same order as files
    site_alpha=0.2,                 # transparency for TFBS boxes
    sharey_signals=False,           # keep default; single axis already shared
    title=None,
    font_scale=1.8                  # 🔸 new: controls all font sizes
):
    import matplotlib.pyplot as plt
    import numpy as np
    from pathlib import Path

    chrom, start, end = region
    x = np.arange(start, end)

    signal_bigwigs = list(signal_bigwigs)
    footprint_bigwigs = [] if footprint_bigwigs is None else list(footprint_bigwigs)

    # Load data
    sig_arrays = [read_bw(p, chrom, start, end) for p in signal_bigwigs]
    fp_arrays  = [read_bw(p, chrom, start, end) for p in footprint_bigwigs]

    # Labels
    if labels is None:
        labels = [Path(p).stem for p in (signal_bigwigs + footprint_bigwigs)]
    else:
        assert len(labels) == len(signal_bigwigs) + len(footprint_bigwigs), "labels length mismatch"

    # Setup font sizes
    base_size = 10 * font_scale
    title_size = 12 * font_scale
    tick_size = 8 * font_scale
    legend_size = 8 * font_scale

    fig, ax = plt.subplots()

    # Plot signals first
    line_objs = []
    for arr, p in zip(sig_arrays, signal_bigwigs):
        line_objs += ax.plot(x, arr, lw=1.2, label=Path(p).stem)

    # Then footprints
    for arr, p in zip(fp_arrays, footprint_bigwigs):
        line_objs += ax.plot(x, arr, lw=1.2, linestyle="--", label=Path(p).stem)

    # Sites as shaded boxes
    if sites_bed:
        sites = load_sites(sites_bed, chrom, start, end)
        y0, y1 = ax.get_ylim()
        for _, r in sites.iterrows():
            ax.axvspan(int(r.start), int(r.end), alpha=site_alpha)

    # Apply labels
    ax.set_xlabel(f"{chrom}:{start:,}-{end:,}", fontsize=base_size, weight="bold")
    ax.set_ylabel("Signal / Footprint (a.u.)", fontsize=base_size, weight="bold")
    ax.tick_params(axis='both', labelsize=tick_size)
    if title:
        ax.set_title(title, fontsize=title_size, weight="bold")

    # Legend
    if labels:
        for line, new_lab in zip(line_objs, labels):
            line.set_label(new_lab)
    ax.legend(frameon=False, ncol=2, fontsize=legend_size)

    plt.tight_layout()
    return ax

In [ ]:
# RAB6B 
mdd_sig = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/InN.5/mdd_corrected_corrected.bw"
ctrl_sig  = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/InN.5/control_corrected_corrected.bw"
mdd_fp  = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/InN.5/footprints/mdd_footprints.bw"
ctrl_fp   = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/InN.5/footprints/control_footprints.bw"
TF_bed  = "/groups/mb928_gp/mp4486/tobias/CURRENT/BINDetect_output/InN.5/KLF15_MA1513.2/beds/KLF15_MA1513.2_all.bed"

region = ("chr3", 133894426, 133896483)

plot_chromatin_tracks(
    region=region,
    signal_bigwigs=[ctrl_sig , mdd_sig],
    footprint_bigwigs=[ctrl_fp, mdd_fp],
    sites_bed=TF_bed,
    labels=["CTRL corrected ATAC-seq", "MDD corrected ATAC-seq", "CTRL footprint score", "MDD footprint score"],
    title="KLF15 RAB6B (MDD vs CTRL)"
)
plt.savefig("KLF15 RAB6B.pdf", bbox_inches="tight")
plt.show()

In [ ]:
# RAB6B detail
mdd_sig = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/InN.5/mdd_corrected_corrected.bw"
ctrl_sig  = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/InN.5/control_corrected_corrected.bw"
mdd_fp  = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/InN.5/footprints/mdd_footprints.bw"
ctrl_fp   = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/InN.5/footprints/control_footprints.bw"
TF_bed  = "/groups/mb928_gp/mp4486/tobias/CURRENT/BINDetect_output/InN.5/KLF15_MA1513.2/beds/KLF15_MA1513.2_all.bed"

region = ("chr3", 133896300, 133896350)

plot_chromatin_tracks(
    region=region,
    signal_bigwigs=[ctrl_sig , mdd_sig],
    footprint_bigwigs=[ctrl_fp, mdd_fp],
    sites_bed=TF_bed,
    labels=["CTRL corrected ATAC-seq", "MDD corrected ATAC-seq", "CTRL footprint score", "MDD footprint score"],
    title="KLF15 RAB6B (MDD vs CTRL)"
)
plt.savefig("KLF15 RAB6B_detail.pdf", bbox_inches="tight")
plt.show()

In [ ]:
# CAMKV
mdd_sig = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/InN.5/mdd_corrected_corrected.bw"
ctrl_sig  = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/InN.5/control_corrected_corrected.bw"
mdd_fp  = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/InN.5/footprints/mdd_footprints.bw"
ctrl_fp   = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/InN.5/footprints/control_footprints.bw"
TF_bed  = "/groups/mb928_gp/mp4486/tobias/CURRENT/BINDetect_output/InN.5/KLF15_MA1513.2/beds/KLF15_MA1513.2_all.bed"

region = ("chr3", 49868961, 49869470)

plot_chromatin_tracks(
    region=region,
    signal_bigwigs=[ctrl_sig , mdd_sig],
    footprint_bigwigs=[ctrl_fp, mdd_fp],
    sites_bed=TF_bed,
    labels=["CTRL corrected ATAC-seq", "MDD corrected ATAC-seq", "CTRL footprint score", "MDD footprint score"],
    title="KLF15 CAMKV (MDD vs CTRL)"
)
plt.savefig("KLF15 CAMKV.pdf", bbox_inches="tight")
plt.show()

In [ ]:
# CAMKV detail
mdd_sig = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/InN.5/mdd_corrected_corrected.bw"
ctrl_sig  = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/InN.5/control_corrected_corrected.bw"
mdd_fp  = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/InN.5/footprints/mdd_footprints.bw"
ctrl_fp   = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/InN.5/footprints/control_footprints.bw"
TF_bed  = "/groups/mb928_gp/mp4486/tobias/CURRENT/BINDetect_output/InN.5/KLF15_MA1513.2/beds/KLF15_MA1513.2_all.bed"

region = ("chr3", 49869300, 49869350)

plot_chromatin_tracks(
    region=region,
    signal_bigwigs=[ctrl_sig , mdd_sig],
    footprint_bigwigs=[ctrl_fp, mdd_fp],
    sites_bed=TF_bed,
    labels=["CTRL corrected ATAC-seq", "MDD corrected ATAC-seq", "CTRL footprint score", "MDD footprint score"],
    title="KLF15 CAMKV (MDD vs CTRL)"
)
plt.savefig("KLF15 CAMKV_detail.pdf", bbox_inches="tight")
plt.show()

In [ ]:
# HPCAL1
mdd_sig = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/GN.1/mdd_corrected_corrected.bw"
ctrl_sig  = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/GN.1/control_corrected_corrected.bw"
mdd_fp  = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/GN.1/footprints/mdd_footprints.bw"
ctrl_fp   = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/GN.1/footprints/control_footprints.bw"
TF_bed  = "/groups/mb928_gp/mp4486/tobias/CURRENT/BINDetect_output/GN.1/KLF15_MA1513.2/beds/KLF15_MA1513.2_all.bed"

region = ("chr2", 10301967, 10303489)

plot_chromatin_tracks(
    region=region,
    signal_bigwigs=[ctrl_sig , mdd_sig],
    footprint_bigwigs=[ctrl_fp, mdd_fp],
    sites_bed=TF_bed,
    labels=["CTRL corrected ATAC-seq", "MDD corrected ATAC-seq", "CTRL footprint score", "MDD footprint score"],
    title="KLF15 HPCAL1 (MDD vs CTRL)"
)
plt.savefig("KLF15 HPCAL1.pdf", bbox_inches="tight")
plt.show()

In [ ]:
# HPCAL1 detail
mdd_sig = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/GN.1/mdd_corrected_corrected.bw"
ctrl_sig  = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/GN.1/control_corrected_corrected.bw"
mdd_fp  = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/GN.1/footprints/mdd_footprints.bw"
ctrl_fp   = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/GN.1/footprints/control_footprints.bw"
TF_bed  = "/groups/mb928_gp/mp4486/tobias/CURRENT/BINDetect_output/GN.1/KLF15_MA1513.2/beds/KLF15_MA1513.2_all.bed"

region = ("chr2", 10303025, 10303075)

plot_chromatin_tracks(
    region=region,
    signal_bigwigs=[ctrl_sig , mdd_sig],
    footprint_bigwigs=[ctrl_fp, mdd_fp],
    sites_bed=TF_bed,
    labels=["CTRL corrected ATAC-seq", "MDD corrected ATAC-seq", "CTRL footprint score", "MDD footprint score"],
    title="KLF15 HPCAL1 (MDD vs CTRL)"
)
plt.savefig("KLF15 HPCAL1_detail.pdf", bbox_inches="tight")
plt.show()

In [ ]:
# KRAS
mdd_sig = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/GN.1/mdd_corrected_corrected.bw"
ctrl_sig  = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/GN.1/control_corrected_corrected.bw"
mdd_fp  = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/GN.1/footprints/mdd_footprints.bw"
ctrl_fp   = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/GN.1/footprints/control_footprints.bw"
TF_bed  = "/groups/mb928_gp/mp4486/tobias/CURRENT/BINDetect_output/GN.1/KLF15_MA1513.2/beds/KLF15_MA1513.2_all.bed"

region = ("chr12", 25249928, 25251660)

plot_chromatin_tracks(
    region=region,
    signal_bigwigs=[ctrl_sig , mdd_sig],
    footprint_bigwigs=[ctrl_fp, mdd_fp],
    sites_bed=TF_bed,
    labels=["CTRL corrected ATAC-seq", "MDD corrected ATAC-seq", "CTRL footprint score", "MDD footprint score"],
    title="KLF15 KRAS (MDD vs CTRL)"
)
plt.savefig("KLF15 KRAS.pdf", bbox_inches="tight")
plt.show()

In [ ]:
# KRAS detail
mdd_sig = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/GN.1/mdd_corrected_corrected.bw"
ctrl_sig  = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/GN.1/control_corrected_corrected.bw"
mdd_fp  = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/GN.1/footprints/mdd_footprints.bw"
ctrl_fp   = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/GN.1/footprints/control_footprints.bw"
TF_bed  = "/groups/mb928_gp/mp4486/tobias/CURRENT/BINDetect_output/GN.1/KLF15_MA1513.2/beds/KLF15_MA1513.2_all.bed"

region = ("chr12", 25250600, 25250650)

plot_chromatin_tracks(
    region=region,
    signal_bigwigs=[ctrl_sig , mdd_sig],
    footprint_bigwigs=[ctrl_fp, mdd_fp],
    sites_bed=TF_bed,
    labels=["CTRL corrected ATAC-seq", "MDD corrected ATAC-seq", "CTRL footprint score", "MDD footprint score"],
    title="KLF15 KRAS (MDD vs CTRL)"
)
plt.savefig("KLF15 KRAS_detail.pdf", bbox_inches="tight")
plt.show()

In [ ]:
# Homer1
mdd_sig = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/ExN.1/mdd_corrected_corrected.bw"
ctrl_sig  = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/ExN.1/control_corrected_corrected.bw"
mdd_fp  = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/ExN.1/footprints/mdd_footprints.bw"
ctrl_fp   = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/ExN.1/footprints/control_footprints.bw"
TF_bed  = "/groups/mb928_gp/mp4486/tobias/CURRENT/BINDetect_output/ExN.1/KLF15_MA1513.2/beds/KLF15_MA1513.2_all.bed"

region = ("chr5", 79513111, 79514912)

plot_chromatin_tracks(
    region=region,
    signal_bigwigs=[ctrl_sig , mdd_sig],
    footprint_bigwigs=[ctrl_fp, mdd_fp],
    sites_bed=TF_bed,
    labels=["CTRL corrected ATAC-seq", "MDD corrected ATAC-seq", "CTRL footprint score", "MDD footprint score"],
    title="KLF15 Homer1 (MDD vs CTRL)"
)
plt.savefig("KLF15 Homer1.pdf", bbox_inches="tight")
plt.show()

In [ ]:
# Homer1 detail
mdd_sig = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/ExN.1/mdd_corrected_corrected.bw"
ctrl_sig  = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/ExN.1/control_corrected_corrected.bw"
mdd_fp  = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/ExN.1/footprints/mdd_footprints.bw"
ctrl_fp   = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/ExN.1/footprints/control_footprints.bw"
TF_bed  = "/groups/mb928_gp/mp4486/tobias/CURRENT/BINDetect_output/ExN.1/KLF15_MA1513.2/beds/KLF15_MA1513.2_all.bed"

region = ("chr5", 79513750, 79513800)

plot_chromatin_tracks(
    region=region,
    signal_bigwigs=[ctrl_sig , mdd_sig],
    footprint_bigwigs=[ctrl_fp, mdd_fp],
    sites_bed=TF_bed,
    labels=["CTRL corrected ATAC-seq", "MDD corrected ATAC-seq", "CTRL footprint score", "MDD footprint score"],
    title="KLF15 HOMER1 (MDD vs CTRL)"
)
plt.savefig("KLF15 Homer1dd_detail.pdf", bbox_inches="tight")
plt.show()

In [ ]:
# SLC25A6
mdd_sig = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/InN.5/mdd_corrected_corrected.bw"
ctrl_sig  = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/InN.5/control_corrected_corrected.bw"
mdd_fp  = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/InN.5/footprints/mdd_footprints.bw"
ctrl_fp   = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/InN.5/footprints/control_footprints.bw"
TF_bed  = "/groups/mb928_gp/mp4486/tobias/CURRENT/BINDetect_output/InN.5/KLF15_MA1513.2/beds/KLF15_MA1513.2_all.bed"

region = ("chrX", 1391530, 1393110)

plot_chromatin_tracks(
    region=region,
    signal_bigwigs=[ctrl_sig , mdd_sig],
    footprint_bigwigs=[ctrl_fp, mdd_fp],
    sites_bed=TF_bed,
    labels=["CTRL corrected ATAC-seq", "MDD corrected ATAC-seq", "CTRL footprint score", "MDD footprint score"],
    title="KLF15 SLC25A6 (MDD vs CTRL)"
)
plt.savefig("KLF15 SLC25A6.pdf", bbox_inches="tight")
plt.show()

In [ ]:
# SLC25A6 detail
mdd_sig = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/InN.5/mdd_corrected_corrected.bw"
ctrl_sig  = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/InN.5/control_corrected_corrected.bw"
mdd_fp  = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/InN.5/footprints/mdd_footprints.bw"
ctrl_fp   = "/groups/mb928_gp/mp4486/tobias/CURRENT/merged_bams/InN.5/footprints/control_footprints.bw"
TF_bed  = "/groups/mb928_gp/mp4486/tobias/CURRENT/BINDetect_output/InN.5/KLF15_MA1513.2/beds/KLF15_MA1513.2_all.bed"

region = ("chrX", 1392000, 1392050)

plot_chromatin_tracks(
    region=region,
    signal_bigwigs=[ctrl_sig , mdd_sig],
    footprint_bigwigs=[ctrl_fp, mdd_fp],
    sites_bed=TF_bed,
    labels=["CTRL corrected ATAC-seq", "MDD corrected ATAC-seq", "CTRL footprint score", "MDD footprint score"],
    title="KLF15 SLC25A6 (MDD vs CTRL)"
)
plt.savefig("KLF15 SLC25A6_detail.pdf", bbox_inches="tight")
plt.show()